In [1]:
import os
import sys

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
from hydra import initialize, compose
import pickle
from glob import glob
from pandeia.engine.calc_utils import build_default_calc, build_default_source
from skimage import restoration
from lenstronomy.Data.psf import PSF
from astropy.convolution import convolve
from tqdm import tqdm
from webbpsf import roman

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='../../config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.lenses.test import SampleSkyPyLens
from mejiro.plots import diagnostic_plot, plot, plot_util, overplot
from mejiro.analysis import stats
from mejiro.utils import util
from mejiro.helpers import pyhalo, lenstronomy_sim, pandeia_input, convert, bkg, psf

In [2]:
output_dir = os.path.join(repo_dir, 'mejiro', 'data', 'default_psfs')

In [3]:
bands = ['F106', 'F129', 'F158', 'F184']
factors = [1, 3, 5]

In [4]:
for factor in tqdm(factors):
    for band in bands:
        wfi = roman.WFI()
        wfi.filter = band
        outfile = os.path.join(output_dir, f'webbpsf_sca01_center_{wfi.filter.lower()}_{factor}.fits')
        wfi.calc_psf(oversample=factor, outfile=outfile, fov_pixels=91) 